# Классификация тем текстов

In [3]:
!pip install lightautoml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.4/416.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from lightautoml.tasks import Task
from lightautoml.automl.presets.text_presets import TabularNLPAutoML

# Препроцессинг данных

In [5]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')
df.sample(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/train.csv'

In [7]:
df.shape

(14919978, 2)

In [8]:
topics = [
    'work', 'news', 'sports', 'music', 'movies', 'politics', 'phones',
    'self-driving cars', 'family', 'cars', 'climate change', 'languages',
    'business', 'health', 'science', 'style', 'opinion', 'economy',
    'history', 'technology', 'affair', 'development', 'mobility'
]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
for topic in topics:
    df[topic] = df['labels'].str.contains(topic).astype(int)

# Train models

In [10]:
task = Task('binary')

text_params = {
    'lang': 'en'
}

# Идея

Библиотека AutoML еще плохо работает с MultiLabel классификацией, поэтому мы решили обучать не 1 большую мультилейбловую модель, а 23 модели бинарной классификации, каждая из которых будет определять - относится текст к данному топику или нет

#### Таким образом мы разбиваем нашу задачу на множество подзадач бинарной классификации

In [11]:
topic = topics[15]
topic

'style'

In [12]:
new_df = df[['text', topic]]

In [22]:
topic_df = new_df.loc[new_df[topic] == 1]
len(topic_df)

3646723

In [23]:
non_topic_df = new_df.loc[new_df[topic] == 0]
len(non_topic_df)

3646723

Берем небольшой батч на обучение так, чтобы было одинаковое количество текстов, одни - которые не подпадают под топик, а другие - попадают

In [24]:
topic_df = topic_df.sample(50000)
non_topic_df = non_topic_df.sample(50000)

In [25]:
new_df = pd.concat([topic_df, non_topic_df], ignore_index=True)

train, test = train_test_split(new_df, random_state=42)

In [26]:
roles = {
    'target': topic,
    'text': 'text'
}

In [27]:
automl = TabularNLPAutoML(
    task=task,
    text_params=text_params
)

In [28]:
train_predictions = automl.fit_predict(train, roles=roles, verbose=0)

INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO3:lightautoml.automl.presets.text_presets:Model language mode: en
INFO:lightautoml.automl.presets.base:Task: binary

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 3600.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 4 cores
INFO:lightautoml.automl.presets.base:- memory: 16 GB

INFO:lightautoml.reader.base:Train data shape: (75000, 2)

INFO:lightautoml.automl.base:Layer 1 train process start. Time left 3599.66 secs
INFO:lightautoml.ml_algo.base:Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
DEBUG:lightautoml.ml_algo.base:Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [], 'embed_sizes': (), 'data_size': 100}
INFO2:lightautoml.ml_algo.base:===== Start working wi

In [1]:
test_predictions = automl.predict(test, batch_size=1000)

NameError: name 'automl' is not defined

In [30]:
print(f'TRAIN SCORE: {roc_auc_score(train[topic].values, train_predictions.data[:, 0])}')
print(f'TEST SCORE: {roc_auc_score(test[topic].values, test_predictions.data[:, 0])}')

TRAIN SCORE: 0.7587999974394297
TEST SCORE: 0.7627892129711551


Неплохо, учитывая, что у нас немного текстов на обучение

# Predict

In [31]:
submission = pd.read_csv('/content/drive/MyDrive/test.csv')
submission

,text
0,noraemingi CaratExolAtiny ATEEZofficial YES QU...
1,Not to mention I take advantage of Steam Cloud...
2,MlindoRSA Dropping tomorrow Yekela NewMusic f...
3,Singh's supporters refused to hand over the b...
4,"Founded in 1981, Bafna Pharmaceutical, which ..."
...,...
3729985,CBSins69 love it n follow me nNaughtySinsatio...
3729986,SOFTJEETH this is still my peak jisung nothin...
3729987,There's interest from developers in Windows P...
3729988,***Seriously it seems like you have played a l...


In [ ]:
preds = automl.predict(submission, batch_size=100_000)

INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:lightautoml.transformers.text:Feature concated__text transformed
INFO3:

И где-то после 30 батча (из 40) модель падала из-за нехватки оперативки, поэтому получить предикты не получилось, однако на обучении при валидации мы набирали 70+ процентов

Думаю, что если бы датасет был поменьше либо если бы у нас были бОльшие вычислительные мощности, то задача бы решилась со скором под 70-80%, но увы :(

Далее можете посмотреть ноутбук с MVP модели суммаризации, модель лежит в ноутбуке, а сама полноценная реализация бота - в другой ветке

In [ ]:
preds